In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../df_new_mean_train.csv")
df.drop(columns=["Unnamed: 0"],inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
0,3.0,58.0,1.0,450.600000,1.640000,450.600000,526.440000,446.500000,5097.950000,-8.700000,...,80062.392773,0.670011,-0.912051,0.018315,242.648806,21.0,21.0,21.0,-1.714518,3.600000
1,3.0,41.0,0.0,357.000000,1.923077,357.076923,538.840237,356.076923,11891.224852,-1.307692,...,76406.068692,0.850813,-0.498918,-0.076761,974.391877,27.0,27.0,27.0,8.932387,-1.307692
2,3.0,76.0,0.0,478.105263,1.041551,479.052632,345.839335,478.210526,2771.745152,0.315789,...,85871.392384,0.489407,-0.973497,0.262626,25.275439,20.0,20.0,20.0,6.346053,0.526316
3,3.0,27.0,1.0,587.066667,357.262222,587.866667,663.182222,584.533333,6410.382222,-0.800000,...,103785.759900,1.091541,-0.392591,-0.159420,9.601546,16.0,16.0,16.0,11.755690,-0.666667
4,2.0,76.0,0.0,900.600000,95.240000,900.600000,170.440000,905.000000,10025.000000,2.800000,...,81079.175405,0.800294,-0.779998,-0.343434,11.737973,11.0,11.0,11.0,-143.141574,2.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8479,0.0,64.0,0.0,612.800000,29298.560000,611.733333,28027.128889,614.933333,28417.528889,-5.866667,...,100877.375033,0.305497,-1.525075,-0.201948,2438.340543,15.0,15.0,15.0,24.802373,8.571429
8480,3.0,75.0,0.0,418.956522,2734.563327,418.956522,2523.780718,419.478261,8352.075614,-3.565217,...,81455.005289,0.448115,-1.260327,0.309823,32.322927,24.0,24.0,24.0,5.947163,-3.565217
8481,1.0,58.0,1.0,1235.428571,700.244898,1235.714286,897.632653,1248.285714,15626.775510,16.857143,...,81763.393500,0.092227,-1.692873,0.599331,10.950366,8.0,8.0,8.0,18.149394,18.571429
8482,2.0,75.0,1.0,823.800000,4775.560000,824.000000,4056.000000,804.000000,11132.800000,-22.600000,...,66478.783260,1.024300,-0.596278,0.147300,335.316688,11.0,11.0,11.0,4.484999,-20.400000


In [3]:
x_train = df.iloc[:,1:].values
y_train = df.iloc[:,0].values

In [4]:
X = x_train
y = y_train

In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X = sc.fit_transform(X)

In [6]:
from sklearn.metrics import confusion_matrix
def confusion_matrix_scorer(clf=None,X=None, y=None,y_pred=None,y_prob=None):
    if clf != None:
        y_pred = clf.predict(X)
        y_prob = clf.predict_proba(X)[:,1]
    cm = confusion_matrix(y,y_pred)
    acc_arr = []
    precision_arr = []
    recall_arr = []
    specificity_arr = []
    f1_arr = []
    for c in range(0,len(cm)):
        c = len(cm)-1
        tp = cm[c,c]
        fp = sum(cm[:,c]) - cm[c,c]
        fn = sum(cm[c,:]) - cm[c,c]
        tn = sum(np.delete(sum(cm)-cm[c,:],c))
        acc_arr.append((tp+tn) / (tp+fn+tn+fp))
        recall_arr.append(tp/(tp+fn))
        precision_arr.append(tp/(tp+fp))
        specificity_arr.append(tn/(tn+fp))
        f1_arr.append(2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn)))))
    return {'acc':np.nanmean(acc_arr),'precision':np.nanmean(precision_arr),'specificity':np.nanmean(specificity_arr),'recall':np.nanmean(recall_arr),'f1_score':np.nanmean(f1_arr)}

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
model = RandomForestClassifier()
params = {
    'n_estimators': [10,50,100,1000],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [3,4,5],
    'max_features':['sqrt', 'log2'],
}
grid_search = GridSearchCV(estimator=model, param_grid=params, cv=10, verbose=2, return_train_score=True,refit=True)
grid_model = grid_search.fit(X,y)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.2s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.2s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.3s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.2s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.3s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.3s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.4s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.4s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.3s
[CV] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=10; total time=   0.4s
[CV] EN

In [8]:
grid_model.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'sqrt',
 'n_estimators': 1000}

In [9]:
grid_model.best_score_

0.9230341745005223

In [10]:
# from sklearn.metrics import ConfusionMatrixDisplay,multilabel_confusion_matrix,f1_score,precision_score,accuracy_score,recall_score,precision_recall_fscore_support
# def evaluation_test(y,y_pred):
#     cm = confusion_matrix(y,y_pred)
#     disp = ConfusionMatrixDisplay(cm,display_labels=['AFIB','SB','SR','GSVT'])
#     disp.plot()
#     plt.show()
#     n_classes = len(cm)
#     result = []
#     for c in range(n_classes):
#         tp = cm[c,c]
#         fp = sum(cm[:,c]) - cm[c,c]
#         fn = sum(cm[c,:]) - cm[c,c]
#         tn = sum(np.delete(sum(cm)-cm[c,:],c))
#         acc = (tp+tn) / (tp+fn+tn+fp)
#         recall = tp/(tp+fn)
#         precision = tp/(tp+fp)
#         specificity = tn/(tn+fp)
#         f1_score = 2*((precision*recall)/(precision+recall))
#         if c+1 == 1:
#             Rhythm = 'AFIB'
#         elif c+1 == 2:
#             Rhythm = 'SB'
#         elif c+1 == 3:
#             Rhythm = 'SR'
#         else:
#             Rhythm = 'GSVT'
#         result.append([Rhythm,acc,recall,precision,f1_score,specificity])
#     p_macro,r_macro,f_macro,support_macro = precision_recall_fscore_support(y,y_pred,average='macro')
#     p_micro,r_micro,f_micro,support_micro = precision_recall_fscore_support(y,y_pred,average='micro')
#     p_weighted,r_weighted,f_weighted,support_weighted = precision_recall_fscore_support(y,y_pred,average='weighted')
#     result.append(['macro avg',None,r_macro,p_macro,f_macro,None])
#     result.append(['micro avg',None,r_micro,p_micro,f_micro,None])
#     result.append(['weighted avg',None,r_weighted,p_weighted,f_weighted,None])
#     return result
        

In [11]:
# evaluation_test = evaluation_test(y_test,result_test)
# df_evaluation_test = pd.DataFrame(data=evaluation_test,columns=["Rhythm Group","ACC","Recall","Precision","F1-score","specificity"])
# df_evaluation_test

In [12]:
# df_evaluation_test.to_csv("../Result/XGB_12_lead_test_mean.csv")